# Pose Estimation from Synthetic Images

Carico l'immagine output del processo di SuperGlue che risulta essere un file npz (una sorta di cartella zip degli array)

In [1]:
import numpy as np
path = "../SuperGluePretrainedNetwork/MoonLu/MoonImg_Out12/Nadir_pitch_15_deg_matches.npz"
#path = "MoonLu/MoonImg_Out/Nadir_pitch_15_deg_matches.npz"
npz = np.load(path)
npz.files
print(npz["matches"])

[335 336  -1 ...  -1  -1  -1]


Sapendo che: 
* Keypoint0 e Keypoint 1 sono i punti di maggior interesse delle due immagini prese in condierazione dal processo di SuperGlue
* il tipo di dati risulta essere un array del tipo (N,2), ovvero N in base al totale numero di punti d'interesse trovati e 2 perchè relativi alla coordinata (x,y) nell'immagine


Una volta presi i vettori di keypoint0 e keypoint1, devo darli in pasto a findEssentialMat() per trovare la matrce essenziale E.
Solo che mi da un errore: OpenCV(4.10.0) D:\bld\libopencv_1726965773712\work\modules\calib3d\src\five-point.cpp:459: error: (-215:Assertion failed) npoints >= 0 && points2.checkVector(2) == npoints && points1.type() == points2.type() in function 'cv::findEssentialMat'
Quindi le strade da percorrere sono due, partendo dal presupposto che il typo di dati in input coincide con quello richiesto dalla funzione:

1- Tagliare semplicemente il vettore di keypoint1 in modo da avere la stessa lunchezza di keypoint0

2- Tagliarlo come in 1., ma riordinarlo anche usando gli indici in "matches"


N.B. la lunghezza focale è da considerarsi in "pixel units", quindi lunghezza_focale/dimensione_pixel; il centro ottico considerato nel centro dell'immagine, quindi risoluzione/2 (uguale sia lungo x che lungo y) 

In [2]:
#METODO 1
coord_1 = npz["keypoints0"]
coord_2 = npz["keypoints1"][0:len(coord_1)]
#print((coord_2))
#print(len(coord_2))
#print(coord_2)

In [14]:
#METODO 2
coord_1 = npz["keypoints0"]
coord_2 = npz["keypoints1"]
#print(coord_2[335])
#print(coord_2[336])
#print(coord_2[2])

#print(len(coord_1))
#print(len(coord_2))
#print(len(npz["matches"]))


for i in range(1,len(npz["matches"])):
    
    if npz["matches"][i-1] != -1:
        index = npz["matches"][i-1]
        #print(index)
        coord_2[i-1] = coord_2[index]

coord_2 = coord_2[0:len(coord_1)]
#print(coord_2[0])
#print(coord_2[1])
#print(coord_2[2])

[261. 342.]
[587. 342.]
[60.  9.]
[261. 342.]
[587. 342.]
[60.  9.]


In [4]:
f = 6.013/(11.7e-3)
#print(f)
c = 1024/2

CameraMat = np.array([
    [f,0,c],
    [0,f,c],
    [0,0,1]
], dtype=np.float32)
#print(type(CameraMat))
#print(npz["matches"])

Per ora stiamo lavorando col metodo 1

Usiamo la fuzione findEssentialMat() per trovare la matrice essenziale utile al processo di RevorePose().
Tale funzione l'ho usata nel modo più semplice (dando i principali input, lascianfo il resto di default; da questo viene fuori che la mask (vettore delle correlazioni) presenta la maggioranza di 0 (ovvero mancata associamento dei punti). Ho provato allora con diversi cambiamenti: usato pesi outdoor (come suggerito da Luca) e ci sono effettivamente molti più Keypoint e associazioni, cambiato il valore del threshold e della prob, ma il problema persiste

Con metodo RANSAC, cambiando la prob, il numero di inlier non aumenta, mentre cambia cambiando il threshold: con threshold 1 ho 64 inlier, mentre threshold 3 ne ho 166.
Con metodo LMEDS , nè probabilitò, nè threshold hanno influenza: ho in tutto 58 inlier, su un vettore lungo 1544 elementi.

In [8]:
#from cv2 import findEssentialMat, recoverPose, RANSAC
import cv2

#E, mask = cv2.findEssentialMat(coord_1, coord_2, CameraMat, method = cv2.RANSAC, prob = 0.999 , threshold = 1.0)
E, mask = cv2.findEssentialMat(coord_1, coord_2, CameraMat)

np.set_printoptions(threshold=np.inf) #Per permettere di visualizzare l'intero vettore a schermo

#print(type(E))
print(sum(mask))
print(len(mask))

[145]
1544


Plot per vedere graficamente come vengono associati i punti delle due immagini

In [6]:
#import matplotlib.pyplot as plt

## Visualizza i punti delle due immagini
#plt.figure(figsize=(10, 5))

## Visualizza i punti della prima immagine
#plt.subplot(1, 2, 1)
#plt.scatter(coord_1[:, 0], coord_1[:, 1], color='blue', marker='o')
#plt.title('Punti coord_1')
#plt.xlim(0, 1024)
#plt.ylim(0, 1024)
#plt.gca().set_aspect('equal', adjustable='box')

## Visualizza i punti della seconda immagine
#plt.subplot(1, 2, 2)
#plt.scatter(coord_2[:, 0], coord_2[:, 1], color='red', marker='o')
#plt.title('Punti coord_2')
#plt.xlim(0, 1024)
#plt.ylim(0, 1024)
#plt.gca().set_aspect('equal', adjustable='box')

#plt.show()

Prima porto i vettori e matrici ad avere lo stesso livello di precisione, per poi usarloi n RevorePose(), ma ottengo l'errore:
OpenCV(4.10.0) D:\bld\libopencv_1726965773712\work\modules\core\src\matmul.dispatch.cpp:356: error: (-215:Assertion failed) type == B.type() in function 'cv::gemm'

In [7]:
E = E.astype(np.float32)
coord_1 = np.ascontiguousarray(coord_1, dtype=np.float32)
coord_2 = np.ascontiguousarray(coord_2, dtype=np.float32)

#Pose, R, T, mask = cv2.recoverPose(E, coord_1, coord_2, CameraMat)
retval, R, T, mask1 = cv2.recoverPose(E, coord_1, coord_2, CameraMat)


error: OpenCV(4.10.0) D:\bld\libopencv_1726965773712\work\modules\core\src\matmul.dispatch.cpp:356: error: (-215:Assertion failed) type == B.type() in function 'cv::gemm'
